 Raw Code to Import Current Price

In [2]:
from yahoofinancials import YahooFinancials
import pandas as pd
import stockquotes
import requests
from bs4 import BeautifulSoup
import ast
import matplotlib.pyplot as plt
import datetime, re

In [4]:
def get_data(symbol):
    # Credit: John Watson Rooney (Youtube)
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.53 Safari/537.36 Edg/103.0.1264.37'}
    url = f'https://finance.yahoo.com/quote/{symbol}'

    r = requests.get(url, headers=header)
    soup = BeautifulSoup(r.text, 'html.parser')

    texts = [span.get_text() for span in soup.find_all()]
    volume = []
    targetPrice = []
    for count, vol in enumerate(texts):
        if vol == 'Volume':
            volume.append(texts[count+1])
        elif vol == '1y Target Est':
            targetPrice.append(texts[count+1])
    time_stamp = datetime.datetime.now() - datetime.timedelta(hours=13)
    time_stamp = time_stamp.strftime("%y-%m-%d %H:%M:%S")
    
    stock = {
        'Time Stamp': time_stamp,
        'Stock': symbol,
        'Price': soup.find('div', {'class': 'D(ib) Mend(20px)'}).find_all()[0].text,
        'Change': soup.find('div', {'class': 'D(ib) Mend(20px)'}).find_all('span')[0].text,
        'Percentage': soup.find('div', {'class': 'D(ib) Mend(20px)'}).find_all('span')[1].text.strip('()'),
        'Volume': int(re.sub(',','',volume[1])),
        '1y Target Est.': targetPrice[1]
    }
    return stock
    
tickers = 'AAPL'
data = get_data(tickers)
df = pd.DataFrame(data, index=[1])
df



,Time Stamp,Stock,Price,Change,Percentage,Volume,1y Target Est.
1,22-07-09 06:24:15,AAPL,147.04,+0.69,+0.47%,63105520,185.47


In [5]:
tickers = ['META', 'AAPL']
i = 0
while(i < 5):
    info = []
    col = []
    
    for ticker in tickers:
        df = pd.DataFrame(get_data(ticker), index=[1])
        df.to_csv(ticker+'.csv', mode='a', header=False)
        df
    i += 1

 StockQuotes Repo ReBuild (Current Data, Historical Data) from Yahoo

In [3]:
data = stockquotes.Stock('META')
current_price = data.current_price
price_change = data.increase_dollars
change_percent = data.increase_percent
print(current_price, price_change, change_percent)
print(type(change_percent), type(price_change), type(current_price))

159.31 -1.94 -1.2
<class 'float'> <class 'float'> <class 'float'>


In [4]:
data = stockquotes.Stock('META')
hist_price = data.historical
df = pd.DataFrame(hist_price)
df.to_csv('META_quotes.csv')
print(df.head(10))



        date    open    high     low   close  adjusted_close    volume
0 2022-07-01  160.31  160.62  155.00  159.31          159.31  24556260
1 2022-06-30  162.15  165.23  158.49  161.25          161.25  35193500
2 2022-06-29  163.57  166.33  160.32  163.94          163.94  28595200
3 2022-06-28  169.90  171.30  160.61  160.68          160.68  27744500
4 2022-06-27  171.32  171.75  168.01  169.49          169.49  29174600
5 2022-06-24  161.73  170.25  161.30  170.16          170.16  61991200
6 2022-06-23  156.27  159.75  154.25  158.75          158.75  40499200
7 2022-06-22  158.38  160.79  155.44  155.85          155.85  47267800
8 2022-06-21  165.93  167.76  155.97  157.05          157.05  50724900
9 2022-06-17  161.68  165.90  159.90  163.74          163.74  31295300


 FMP Cloud API

In [9]:
import os
from dotenv import load_dotenv
import typing
import fmpsdk

api = '81c23a3c274fb42826c3834f2601b5e3'

# Company Valuation Methods
symbol: str = "META"
print(f"Company Profile: {fmpsdk.company_profile(apikey=api, symbol=symbol)}")


Company Profile: [{'symbol': 'META', 'price': 160.03, 'beta': 1.376134, 'volAvg': 33100198, 'mktCap': 433092362240, 'lastDiv': 0.0, 'range': '154.25-384.33', 'changes': -1.2200012, 'companyName': 'Meta Platforms, Inc.', 'currency': 'USD', 'cik': '0001326801', 'isin': 'US30303M1027', 'cusip': '30303M102', 'exchange': 'NASDAQ Global Market', 'exchangeShortName': 'NASDAQ', 'industry': 'Internet Content & Information', 'website': 'https://investor.fb.com', 'description': "Meta Platforms, Inc. develops products that enable people to connect and share with friends and family through mobile devices, personal computers, virtual reality headsets, wearables, and in-home devices worldwide. It operates in two segments, Family of Apps and Reality Labs. The Family of Apps segment's products include Facebook, which enables people to share, discover, and connect with interests; Instagram, a community for sharing photos, videos, and private messages, as well as feed, stories, reels, video, live, and sh